In [184]:
import networkx as nx
import numpy as np
from matplotlib import pyplot as plt
from functools import partial

def filter_edge(capacity, g):
    graph = nx.to_networkx_graph(g).edges()
    def filter_func(pair, graph):
        ed = set(graph)
        k, _ = pair
        if k in ed:
            return True
        else:
            return False
    filter_f = partial(filter_func, graph=graph)
    return dict(filter(filter_f, capacity.items()))

def generate_graph(size=10, nodes=10, edges=20, capacity=None):

    if capacity is None:
        capacity = {}
        G = nx.complete_graph(nodes)
        for edge in G.edges:
            capacity[edge] =  randint(60, 300)
    graph = np.array([nx.to_numpy_array(nx.gnm_random_graph(n=nodes, m=edges)) for _ in range(size)])
    graph_cap = np.array([list(filter_edge(capacity, G).values()) for G in graph])
    return graph, graph_cap, capacity

In [ ]:
from numpy.random import choice, randint, random

def clone_ind(size, pCl):
    idx = choice(a=size, size = int(np.ceil(pCl*size)), replace=False)
    return idx

In [187]:

def mutate_outga(individual, global_cap, pMu=0.2):
    if random() > pMu:
        return False
    G = nx.to_networkx_graph(individual)
    edges = list(G.edges())
    n_nodes = G.number_of_nodes()
    edge = edges[choice(a=len(edges))]
    s, t = edge
    exclusion = [s, t]
    source = choice(exclusion)
    for s, t in edges:
        if s == source:
            exclusion.append(t)
        elif t == source:
            exclusion.append(s)

    i = choice(a=[i for i in range(n_nodes) if i not in exclusion ])
    ind = individual.copy()
    ind[s][t] = 0
    ind[t][s] = 0

    ind[source][i] = 1
    ind[i][source] = 1

    graph_cap = np.array([list(filter_edge(global_cap, ind).values())])

    return ind, graph_cap


In [189]:

def crossover_outga(parent1, parent2, global_cap, edge_size,pCr=0.8):
    if random() > pCr:
        return False
    p1, p2 = parent1.copy(), parent2.copy()
    chrom_len = len(p1)
    temp = (p1 + p2) / 2

    redundant = np.floor(temp)
    new_pair = np.ceil(temp - redundant)
    total_pair = redundant + new_pair

    redundant_sum = redundant.sum()/2
    np_sum = new_pair.sum()/2
    current_sum = redundant_sum + np_sum

    final_top = total_pair
    if current_sum > edge_size:
        nr_edge = int(current_sum - edge_size)
        if nr_edge == redundant_sum:
            final_top = new_pair
        elif nr_edge < redundant_sum:
            G = nx.to_networkx_graph(redundant)
            edges = np.array(G.edges())
            edge = edges[choice(a=len(edges), size=nr_edge,replace=False)]
            for s, t in edge:
                redundant[s][t] = 0
                redundant[t][s] = 0
            final_top = redundant + new_pair
        elif nr_edge > redundant_sum:
            G = nx.to_networkx_graph(total_pair)
            edges = np.array(G.edges())
            edge = edges[choice(a=len(edges), size=nr_edge,replace=False)]
            for s, t in edge:
                total_pair[s][t] = 0
                total_pair[t][s] = 0
            final_top = total_pair

    graph_cap = np.array([list(filter_edge(global_cap, final_top).values())])
    return final_top, graph_cap


In [165]:
def fitness_outga(individual, flow):
    G = nx.to_networkx_graph(individual)
    min_val, max_val = np.min(flow), np.max(flow)
    scaled = (flow - min_val) / (max_val - min_val)
    for (i, (u,v)) in enumerate(G.edges):
        G[u][v]['flow'] = scaled[i]
    T = nx.average_shortest_path_length(G, weight="flow", method='dijkstra')
    return 1/T
    

In [ ]:

def elitism_selection(size: int, population, fitness: np.array):
    strongest = np.argsort(fitness)[::-1][:size]
    return population[strongest], strongest


In [ ]:

def outerga(size=10, nodes=10, edges=20, generasi=100, capacity=None, pCl=0.4, pMu=0.2, pCr=0.8, innerGAsize=50, innerGAgen=100):
    pop, pop_cap, global_cap = generate_graph(size=size, nodes=nodes, edges=edges)

    # inner Ga
    def get_maxFlow(capacity):
        pop, fit = innerga(size=innerGAsize, generasi=innerGAgen, capacity=capacity)
        fittest = np.argmax(fit)
        flow = pop[fittest]
        return flow

    for gen in range(generasi):
        pop_flow = np.array([get_maxFlow(ind_cap) for ind_cap in pop_cap])

        fitness = np.array([fitness_outga(individual=ind, flow=ind_flow) for ind, ind_flow in zip(pop, pop_flow)])

        nMu = int(np.ceil(size * pMu))
        nCr = int(np.ceil(size * pCr))
        total_select = nMu + nCr

        selected, _ = elitism_selection(size=total_select,population=pop, fitness=fitness)
        idx = clone_ind(size=size, pCl=pCl)
        offspring = np.array(pop[idx].reshape((-1, nodes,nodes)))
        offspring_cap = np.array(pop_cap[idx].reshape((-1, edges)))

        for p1, p2 in zip(selected[:nCr:2], selected[1:nCr:2]):
            if cross := crossover_outga(parent1=p1, parent2=p2,global_cap=global_cap,edge_size=edges, pCr=1):
                child, child_cap = cross
                offspring = np.append(offspring,child.reshape(1, nodes,nodes), axis=0)
                offspring_cap = np.append(offspring_cap, child_cap.reshape(1, edges), axis=0)


        for m in selected[nCr:]:
            mut, mut_cap = mutate_outga(individual=m, global_cap=global_cap,pMu=1)
            offspring = np.append(offspring, mut.reshape((1, nodes,nodes)), axis=0)
            offspring_cap = np.append(offspring_cap, mut_cap.reshape(1, edges), axis=0)

        pop = offspring
        pop_cap = offspring_cap

    last_flow = np.array([get_maxFlow(ind_cap) for ind_cap in pop_cap])
    return pop, np.array([fitness_outga(individual=ind, flow=ind_flow) for ind, ind_flow in zip(pop, last_flow)])
